## ⚙️ SETUP
---
Instalação de bibliotecas e importação de dados

### Importação de dados do GitHub:

In [ ]:
%pip install -q gitpython

import os
import shutil
import git
import tempfile
from pathlib import Path

REPO_URL = "https://github.com/GTazz/CS-Graduation.git"
FOLDER_NAME = "2-Semestre/3 - SO/TrabalhoArquivos/prontuarios"

destination_dir = Path(os.getcwd()) / Path(FOLDER_NAME).name

if not os.path.exists(destination_dir):
    with tempfile.TemporaryDirectory() as tmp:
        git.Repo.clone_from(REPO_URL, tmp, depth=1)

        target_dir = Path(tmp) / FOLDER_NAME
        
        shutil.copytree(target_dir, destination_dir, dirs_exist_ok=True)
            
        print("Arquivos do repositório mesclados no diretório atual.")

### Criação de função de geração de árvore:

In [ ]:
%pip install -q matplotlib

import os
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
from PIL import Image

# ---- Helpers to build and layout the tree (width-aware, no overlap) ----
def build_fs_tree(root, base_path=None):
    if base_path is None:
        base_path = root
    name = os.path.basename(os.path.normpath(root)) or root
    node = {"name": name, "is_dir": True, "children": [], "full_path": root}
    try:
        entries = sorted(os.listdir(root))
    except FileNotFoundError:
        raise FileNotFoundError(f"Caminho não encontrado: {root}")
    for item in entries:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            node["children"].append(build_fs_tree(path, base_path))
        else:
            node["children"].append({"name": item, "is_dir": False, "children": [], "full_path": path})
    return node

def annotate_sizes(node, char_w=0.14, base_w=0.8, min_w=1.3, box_h=0.8, sep=0.6):
    """Annotate each node with box_w/box_h and subtree_w using children widths."""
    label = node["name"] + ("/" if node.get("is_dir", False) else "")
    node["label"] = label

    # Check if it's a JPEG file
    node["is_image"] = node["name"].lower().endswith((".jpeg", ".jpg"))

    # Increase box height for images to accommodate image + caption
    actual_box_h = box_h * 2.0 if node["is_image"] else box_h

    node["box_w"] = max(min_w, base_w + char_w * len(label))
    node["box_h"] = actual_box_h
    for c in node.get("children", []):
        annotate_sizes(c, char_w, base_w, min_w, box_h, sep)
    if node.get("children"):
        total_children = sum(c["subtree_w"] for c in node["children"]) + sep * (len(node["children"]) - 1)
        node["subtree_w"] = max(node["box_w"], total_children)
    else:
        node["subtree_w"] = node["box_w"]
    return node

def assign_positions(node, x_left=0.0, level=0, y_spacing=2.2, sep=0.6):
    """Assign x,y positions so sibling subtrees are spaced by subtree width (no overlap)."""
    node["y"] = -level * y_spacing
    if not node.get("children"):
        node["x"] = x_left + node["box_w"] / 2.0
        return node["x"]
    cursor = x_left
    for i, c in enumerate(node["children"]):
        assign_positions(c, cursor, level + 1, y_spacing, sep)
        cursor += c["subtree_w"] + (sep if i < len(node["children"]) - 1 else 0.0)
    span_left = x_left
    span_right = x_left + sum(c["subtree_w"] for c in node["children"]) + sep * (len(node["children"]) - 1)
    node["x"] = (span_left + span_right) / 2.0
    return node["x"]

def count_depth(node):
    if not node.get("children"):
        return 1
    return 1 + max(count_depth(c) for c in node["children"])

def find_upwards(start_dir, name):
    p = os.path.abspath(start_dir)
    while True:
        candidate = os.path.join(p, name)
        if os.path.exists(candidate):
            return candidate
        parent = os.path.dirname(p)
        if parent == p:
            return None
        p = parent

def resolve_root(root_name):
    # 1. If path exists directly (relative or absolute)
    if os.path.exists(root_name):
        return os.path.abspath(root_name)
    # 2. Try inside a data/ folder upward
    data_base = find_upwards(os.getcwd(), "data")
    if data_base:
        candidate = os.path.join(data_base, root_name)
        if os.path.exists(candidate):
            return candidate
    # 3. Try upward search for folder name itself
    return find_upwards(os.getcwd(), root_name)


def _content_areas(x, y, w, h, padding_frac=0.08, image_frac=0.70):
    pad_x = w * padding_frac
    pad_y = h * padding_frac
    left = x - w/2 + pad_x
    right = x + w/2 - pad_x
    top = y + h/2 - pad_y
    bottom = y - h/2 + pad_y
    content_w = right - left
    content_h = top - bottom
    img_area_h = content_h * image_frac
    txt_area_h = content_h - img_area_h
    return left, right, top, bottom, content_w, content_h, img_area_h, txt_area_h


def _draw_image_inside_box(ax, img, box_patch, x, y, w, h, padding_frac=0.08, image_frac=0.70):
    # Compute inner areas
    left, right, top, bottom, content_w, content_h, img_area_h, txt_area_h = _content_areas(x, y, w, h, padding_frac, image_frac)
    # Preserve aspect ratio to fit into (content_w x img_area_h)
    aspect = img.height / img.width if img.width else 1.0
    w_by_width = content_w
    h_by_width = w_by_width * aspect
    if h_by_width <= img_area_h:
        img_w = w_by_width
        img_h = h_by_width
    else:
        img_h = img_area_h
        img_w = img_h / aspect
    img_xmin = x - img_w/2
    img_xmax = x + img_w/2
    img_ymax = top
    img_ymin = img_ymax - img_h
    ax.imshow(img, extent=[img_xmin, img_xmax, img_ymin, img_ymax], zorder=3, clip_path=box_patch, clip_on=True)
    caption_center_y = bottom + txt_area_h / 2
    return caption_center_y


def draw_tree_auto(root_name):
    base = resolve_root(root_name)
    if base is None:
        print(f"Pasta '{root_name}' não encontrada a partir de {os.getcwd()}.")
        return
    tree = build_fs_tree(base)
    tree = annotate_sizes(tree)
    assign_positions(tree)
    depth = count_depth(tree)
    fig_w = max(10, tree["subtree_w"] * 0.9 + 2)
    fig_h = max(4, depth * 1.7)
    fig, ax = plt.subplots(figsize=(fig_w, fig_h))

    # Edges
    stack = [tree]
    while stack:
        n = stack.pop()
        for c in n.get("children", []):
            ax.plot([n["x"], c["x"]], [n["y"] - 0.35, c["y"] + 0.35], color="#4e555b", linewidth=1.2, zorder=1)
            stack.append(c)

    # Nodes (boxes)
    stack = [tree]
    while stack:
        n = stack.pop()
        x, y = n["x"], n["y"]
        w, h = n["box_w"], n["box_h"]
        face = "#1976d2" if n.get("is_dir", False) else "#9e9e9e"
        box = FancyBboxPatch((x - w/2, y - h/2), w, h,
                             boxstyle="round,pad=0.02,rounding_size=0.08",
                             linewidth=1.0, edgecolor="#2f3941", facecolor=face,
                             alpha=0.95, zorder=2)
        ax.add_patch(box)

        if n.get("is_image", False):
            try:
                img_path = n.get("full_path")
                img = Image.open(img_path).convert("RGBA")
                caption_y = _draw_image_inside_box(ax, img, box, x, y, w, h,
                                                    padding_frac=0.08, image_frac=0.70)
                ax.text(x, caption_y, n["label"], color="white", ha="center", va="center",
                        fontsize=9, family="monospace", zorder=3, clip_path=box, clip_on=True)
            except Exception:
                # Compute areas and place caption at bottom even on failure; draw placeholder in image area
                left, right, top, bottom, content_w, content_h, img_area_h, txt_area_h = _content_areas(x, y, w, h, 0.08, 0.70)
                icon_y = top - img_area_h/2
                ax.text(x, icon_y, "🖼️", color="white", ha="center", va="center",
                        fontsize=14, zorder=3, clip_path=box, clip_on=True)
                caption_y = bottom + txt_area_h/2
                ax.text(x, caption_y, n["label"], color="white", ha="center", va="center",
                        fontsize=9, family="monospace", zorder=3, clip_path=box, clip_on=True)
        else:
            # Normal text centered
            ax.text(x, y, n["label"], color="white", ha="center", va="center",
                    fontsize=10, family="monospace", zorder=3, clip_path=box, clip_on=True)

        stack.extend(n.get("children", []))

    ax.set_xlim(-0.5, tree["subtree_w"] + 0.5)
    ax.set_ylim(-depth * 2.2 + 0.8, 0.8)
    ax.set_title(f"Estrutura em árvore – diretório '{os.path.basename(base)}'", fontsize=14, weight="bold")
    ax.axis("off")
    ax.margins(x=0.02, y=0.02)
    plt.tight_layout()
    plt.show()

# # ---- Run ----
# draw_tree_auto("prontuarios")

### Criação de função de geração de tabela e gráfico:

In [ ]:
%pip install -q pandas jinja2

import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt

def list_files_as_dataframe(root_path):
    """
    Percorre recursivamente um diretório e retorna um DataFrame categorizado com informações dos arquivos.
    
    Retorna DataFrame com colunas organizadas por hierarquia:
    - Status Paciente: Ativo ou Arquivado
    - Nome Paciente: ID e nome do paciente
    - Categoria: Tipo de documento (Perfil, Consultas, Exames, Tratamentos)
    - Nome: nome do arquivo/diretório
    - Tipo: 'Diretório' ou 'Arquivo'
    - Extensão: extensão do arquivo
    - Tamanho (KB): tamanho do arquivo em KB
    - Caminho Completo: caminho relativo completo
    """
    base = resolve_root(root_path)
    if base is None:
        print(f"Pasta '{root_path}' não encontrada.")
        return None
    
    data = []
    base_path = Path(base)
    
    # Percorrer recursivamente
    for item in sorted(base_path.rglob('*')):
        rel_path = item.relative_to(base_path)
        parts = rel_path.parts
        
        # Determinar categorias baseado na estrutura de pastas
        status_paciente = ''
        nome_paciente = ''
        categoria = ''
        
        if len(parts) > 0:
            # Primeiro nível: pacientes_ativos ou pacientes_arquivados
            if 'pacientes_ativos' in parts[0]:
                status_paciente = 'Ativo'
            elif 'pacientes_arquivados' in parts[0]:
                status_paciente = 'Arquivado'
            
            # Segundo nível: nome do paciente (ex: P0-Olavo_de_Carvalho)
            if len(parts) > 1:
                nome_paciente = parts[1]
                
                # Terceiro nível: categoria do documento
                if len(parts) > 2:
                    categoria_raw = parts[2]
                    # Mapear para nomes mais legíveis
                    categoria_map = {
                        'consultas': 'Consultas',
                        'exames': 'Exames',
                        'tratamentos': 'Tratamentos',
                        'cirurgias': 'Cirurgias'
                    }
                    categoria = categoria_map.get(categoria_raw, categoria_raw.capitalize())
                elif item.is_file():
                    # Arquivo diretamente na pasta do paciente (ex: profile.jpg)
                    categoria = 'Perfil'
        
        if item.is_dir():
            data.append({
                'Status Paciente': status_paciente,
                'Nome Paciente': nome_paciente,
                'Categoria': categoria,
                'Nome': item.name,
                'Tipo': 'Diretório',
                'Extensão': '',
                'Tamanho (KB)': '',
                'Caminho Completo': str(rel_path)
            })
        else:
            size = item.stat().st_size
            size_kb = round(size / 1024, 2) if size > 0 else 0
            ext = item.suffix
            data.append({
                'Status Paciente': status_paciente,
                'Nome Paciente': nome_paciente,
                'Categoria': categoria,
                'Nome': item.name,
                'Tipo': 'Arquivo',
                'Extensão': ext,
                'Tamanho (KB)': size_kb,
                'Caminho Completo': str(rel_path)
            })
    
    df = pd.DataFrame(data)
    return df

def save_and_display_csv(root_path, csv_filename='arquivos_listagem.csv'):
    """
    Gera DataFrame categorizado, salva como CSV e exibe no notebook com agrupamento.
    """
    df = list_files_as_dataframe(root_path)
    if df is None:
        return
    
    # Ordenar por categorias lógicas
    df = df.sort_values(
        by=['Status Paciente', 'Nome Paciente', 'Categoria', 'Tipo', 'Nome'],
        ascending=[True, True, True, False, True]  # Diretórios antes de Arquivos
    ).reset_index(drop=True)
    
    # Salvar como CSV
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"✓ Arquivo CSV salvo: {csv_filename}")
    
    # Criar estilo neutro e profissional
    styled = df.style.set_properties(**{
        'text-align': 'left',
        'font-size': '10pt',
        'border': '1px solid #ddd',
        'padding': '8px',
        'color': '#333333'
    }).set_table_styles([
        {'selector': 'th', 'props': [
            ('background-color', '#f5f5f5'), 
            ('color', '#333333'), 
            ('font-weight', 'bold'), 
            ('text-align', 'left'),
            ('padding', '10px'),
            ('border', '1px solid #ddd'),
            ('position', 'sticky'),
            ('top', '0')
        ]},
        {'selector': 'tr:nth-of-type(even)', 'props': [('background-color', '#fafafa')]},
        {'selector': 'tr:nth-of-type(odd)', 'props': [('background-color', '#ffffff')]},
        {'selector': 'tr:hover', 'props': [('background-color', '#f0f0f0')]},
        {'selector': 'td', 'props': [('border', '1px solid #e0e0e0')]}
    ]).hide(axis='index')
    
    # Destacar diretórios vs arquivos com cores neutras
    def color_by_type(val):
        if val == 'Diretório':
            return 'background-color: #e8e8e8; font-weight: bold; color: #333333'
        elif val == 'Arquivo':
            return 'background-color: #ffffff; color: #333333'
        return ''
    
    styled = styled.map(color_by_type, subset=['Tipo'])
    
    return styled

def gerar_relatorio_contagem_categoria(csv_path='mapeamento_prontuarios.csv', mostrar_grafico=True, incluir_perfil=False):
    """Gera contagem de arquivos por categoria a partir do CSV.
    Retorna uma Series com as contagens ordenadas.
    
    Parâmetros:
      csv_path: caminho do CSV previamente gerado (por save_and_display_csv)
      mostrar_grafico: se True, exibe gráfico de barras com as contagens
      incluir_perfil: se False (padrão), exclui categoria 'Perfil' da contagem/gráfico
    """
    if not os.path.exists(csv_path):
        print(f'❌ Arquivo CSV não encontrado: {csv_path}')
        print('Execute primeiro save_and_display_csv() para gerar o arquivo.')
        return None
    
    df = pd.read_csv(csv_path)

    if 'Tipo' in df.columns:
        df_files = df[df['Tipo'] == 'Arquivo'].copy()
    else:
        df_files = df.copy()

    if 'Categoria' not in df_files.columns:
        print("Coluna 'Categoria' não encontrada.")
        return None

    df_files['Categoria'] = df_files['Categoria'].fillna('').replace('', 'Sem categoria')

    # Excluir 'Perfil' ou 'Profile' por padrão (case-insensitive)
    if not incluir_perfil:
        df_files = df_files[~df_files['Categoria'].str.lower().isin(['perfil', 'profile'])]

    counts = (
        df_files.groupby('Categoria')
        .size()
        .sort_values(ascending=False)
    )

    if counts.empty:
        print('Nenhuma categoria com arquivos encontrada.')
        return counts

    if mostrar_grafico:
        plt.figure(figsize=(8, 4))
        counts.plot(kind='bar', color='#1976d2', edgecolor='#2f3941')
        plt.title('Quantidade de arquivos por categoria')
        plt.ylabel('Nº de arquivos')
        plt.xlabel('Categoria')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
    return counts


### Criação de função de geração de árvore de caracteres:

In [ ]:
import os

def list_directory_tree(root_path, prefix=""):
    """
    Recursively prints the directory tree structure starting from root_path.

    :param root_path: Path to the directory to list.
    :param prefix: Used internally for indentation.
    """
    slash = "/" if "/" not in root_path else ""
    print(root_path + slash, end="")
    _list_directory_tree(root_path, prefix)

def _list_directory_tree(root_path, prefix=""):
    try:
        # Validate if path exists and is a directory
        if not os.path.exists(root_path):
            print(f"Error: Path '{root_path}' does not exist.")
            return
        if not os.path.isdir(root_path):
            print(f"Error: Path '{root_path}' is not a directory.")
            return

        # Get sorted list of entries
        entries = sorted(os.listdir(root_path))
        entries_count = len(entries)

        for index, entry in enumerate(entries):
            path = os.path.join(root_path, entry)
            connector = "├── " if index < entries_count - 1 else "└── "
            print("\n", prefix + connector + entry, end="")

            # If entry is a directory, recurse into it
            if os.path.isdir(path):
                print("/", end="")
                extension = "│   " if index < entries_count - 1 else "    "
                _list_directory_tree(path, prefix + extension)

    except PermissionError:
        print(prefix + "└── [Permission Denied]")
    except Exception as e:
        print(f"Unexpected error: {e}")

### Criação de função de Gerenciamento de usuários:

In [ ]:
import os
import shutil
from pathlib import Path
from datetime import datetime
from PIL import Image, ImageDraw

class GerenciadorProntuarios:
    def __init__(self, root_dir="prontuarios"):
        self.root = Path(root_dir)
        
        # Criar estrutura base se não existir
        self._garantir_estrutura()
    
class GerenciadorProntuarios:
    def __init__(self, root_dir="prontuarios"):
        self.root = Path(root_dir)
        
        # Criar estrutura base se não existir
        self._garantir_estrutura()
    
    def _garantir_estrutura(self):
        """Garante que a estrutura de diretórios existe"""
        ativos = self.root / "pacientes_ativos"
        arquivados = self.root / "pacientes_arquivados"
        ativos.mkdir(parents=True, exist_ok=True)
        arquivados.mkdir(parents=True, exist_ok=True)
    
    @property
    def ativos(self):
        """Retorna o caminho para pacientes ativos (sempre atualizado)"""
        return self.root / "pacientes_ativos"
    
    @property
    def arquivados(self):
        """Retorna o caminho para pacientes arquivados (sempre atualizado)"""
        return self.root / "pacientes_arquivados"
    
    def _proximo_id(self):
        """Obtém o próximo ID disponível para pacientes (considerando ativos E arquivados)"""
        ids = []
        
        # Verificar em ambas as pastas (sempre lê do disco)
        for pasta in [self.ativos, self.arquivados]:
            if pasta.exists():
                for item in pasta.iterdir():
                    if item.is_dir() and item.name.startswith('P'):
                        try:
                            id_num = int(item.name.split('-')[0][1:])
                            ids.append(id_num)
                        except:
                            continue
        
        return max(ids) + 1 if ids else 0
    
    def criar_paciente(self, nome, categorias=['consultas', 'exames', 'tratamentos']):
        """
        Cria um novo paciente ativo com estrutura completa.
        
        Args:
            nome: Nome do paciente (ex: "Maria_Silva")
            categorias: Lista de categorias de prontuários a criar
        
        Returns:
            Path do diretório do paciente criado
        """
        next_id = self._proximo_id()
        paciente_dir = self.ativos / f"P{next_id}-{nome}"
        
        if paciente_dir.exists():
            print(f"⚠️  Paciente '{nome}' já existe.")
            return paciente_dir
        
        # Criar pasta do paciente
        paciente_dir.mkdir(parents=True)
        
        # Criar pasta profile
        profile_dir = paciente_dir / "profile"
        profile_dir.mkdir()
        
        # Criar imagem placeholder no profile
        self._criar_imagem_placeholder(profile_dir, f"P{next_id}", nome)
        
        # Criar categorias de prontuários
        for categoria in categorias:
            (paciente_dir / categoria).mkdir()
        
        print(f"✅ Paciente 'P{next_id}-{nome}' criado com sucesso!")
        print(f"   Localização: {paciente_dir}")
        print(f"   Categorias: {', '.join(categorias)}")
        return paciente_dir
    
    def _criar_imagem_placeholder(self, profile_dir, id_paciente, nome):
        """Cria uma imagem placeholder minimalista com ícone de pessoa cinza"""
        # Criar imagem 400x400 com fundo cinza claro
        img = Image.new('RGB', (400, 400), color='#e0e0e0')
        draw = ImageDraw.Draw(img)
        
        # Cor cinza para o ícone
        gray = '#9e9e9e'
        
        # Desenhar cabeça (círculo)
        head_radius = 50
        head_center = (200, 140)
        draw.ellipse([
            head_center[0] - head_radius, 
            head_center[1] - head_radius,
            head_center[0] + head_radius, 
            head_center[1] + head_radius
        ], fill=gray)
        
        # Desenhar corpo (trapézio simples)
        shoulder_width = 140
        body_bottom_width = 180
        body_top = 200
        body_bottom = 350
        
        # Desenhar como um trapézio
        body_points = [
            (200 - shoulder_width//2, body_top),  # Topo esquerdo
            (200 + shoulder_width//2, body_top),  # Topo direito
            (200 + body_bottom_width//2, body_bottom),  # Base direita
            (200 - body_bottom_width//2, body_bottom),  # Base esquerda
        ]
        draw.polygon(body_points, fill=gray)
        
        # Salvar imagem
        img_path = profile_dir / "profile.jpg"
        img.save(img_path, 'JPEG', quality=95)
        print(f"   📷 Imagem de perfil criada: {img_path.name}")
    
    def arquivar_paciente(self, id_paciente):
        """
        Move um paciente de ativos para arquivados.
        
        Args:
            id_paciente: ID numérico do paciente (ex: 3 para "P3-...")
        
        Returns:
            Path do novo local ou None se não encontrado
        """
        # Buscar paciente nos ativos pelo ID
        paciente_dir = self._encontrar_paciente_por_id(self.ativos, id_paciente)
        if not paciente_dir:
            print(f"❌ Paciente com ID {id_paciente} não encontrado em pacientes ativos.")
            return None
        
        # Mover para arquivados
        destino = self.arquivados / paciente_dir.name
        shutil.move(str(paciente_dir), str(destino))
        
        print(f"🗄️  Paciente '{paciente_dir.name}' arquivado com sucesso!")
        print(f"   De: {paciente_dir}")
        print(f"   Para: {destino}")
        return destino
    
    def desarquivar_paciente(self, id_paciente):
        """
        Move um paciente de arquivados para ativos.
        
        Args:
            id_paciente: ID numérico do paciente (ex: 3 para "P3-...")
        
        Returns:
            Path do novo local ou None se não encontrado
        """
        # Buscar paciente nos arquivados pelo ID
        paciente_dir = self._encontrar_paciente_por_id(self.arquivados, id_paciente)
        if not paciente_dir:
            print(f"❌ Paciente com ID {id_paciente} não encontrado em pacientes arquivados.")
            return None
        
        # Mover para ativos
        destino = self.ativos / paciente_dir.name
        shutil.move(str(paciente_dir), str(destino))
        
        print(f"✅ Paciente '{paciente_dir.name}' reativado com sucesso!")
        print(f"   De: {paciente_dir}")
        print(f"   Para: {destino}")
        return destino
    
    def criar_documento(self, id_paciente, categoria, nome_arquivo, conteudo="", arquivado=False):
        """
        Cria um arquivo de documento para um paciente em uma categoria específica.
        
        Args:
            id_paciente: ID numérico do paciente (ex: 3 para "P3-...")
            categoria: Categoria do documento (ex: 'exames', 'consultas', 'cirurgias', 'tratamentos')
            nome_arquivo: Nome do arquivo (ex: "hemograma_2025-11-13.txt")
            conteudo: Conteúdo do arquivo (opcional)
            arquivado: Se True, busca em pacientes_arquivados
        
        Returns:
            Path do arquivo criado ou None se paciente não encontrado
        """
        # Buscar paciente pelo ID
        pasta_busca = self.arquivados if arquivado else self.ativos
        paciente_dir = self._encontrar_paciente_por_id(pasta_busca, id_paciente)
        
        if not paciente_dir:
            status = "arquivados" if arquivado else "ativos"
            print(f"❌ Paciente com ID {id_paciente} não encontrado em pacientes {status}.")
            return None
        
        # Criar pasta da categoria se não existir
        categoria_dir = paciente_dir / categoria
        categoria_dir.mkdir(exist_ok=True)
        
        # Criar arquivo de documento
        arquivo_path = categoria_dir / nome_arquivo
        with open(arquivo_path, 'w', encoding='utf-8') as f:
            if conteudo:
                f.write(conteudo)
            else:
                f.write(f"Documento criado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        print(f"📋 Documento criado com sucesso!")
        print(f"   Paciente: {paciente_dir.name}")
        print(f"   Categoria: {categoria}")
        print(f"   Arquivo: {arquivo_path}")
        return arquivo_path
    
    def _encontrar_paciente(self, pasta, nome_ou_id):
        """Busca um paciente por nome ou ID em uma pasta"""
        for item in pasta.iterdir():
            if item.is_dir():
                # Busca por ID (ex: "P3")
                if item.name.startswith(nome_ou_id + "-"):
                    return item
                # Busca por nome (ex: "Maria_Silva")
                if nome_ou_id in item.name:
                    return item
        return None
    
    def _encontrar_paciente_por_id(self, pasta, id_paciente):
        """Busca um paciente por ID numérico em uma pasta"""
        prefixo = f"P{id_paciente}-"
        for item in pasta.iterdir():
            if item.is_dir() and item.name.startswith(prefixo):
                return item
        return None
    
    def listar_pacientes(self, arquivado=False):
        """Lista todos os pacientes (ativos ou arquivados)"""
        pasta = self.arquivados if arquivado else self.ativos
        status = "Arquivados" if arquivado else "Ativos"
        
        pacientes = [p.name for p in pasta.iterdir() if p.is_dir()]
        
        print(f"\n{'='*50}")
        print(f"Pacientes {status}: {len(pacientes)}")
        print(f"{'='*50}")
        for p in sorted(pacientes):
            print(f"  • {p}")
        print(f"{'='*50}\n")
        
        return pacientes

# Inicializar gerenciador
gp = GerenciadorProntuarios()
print("✅ Gerenciador de Prontuários inicializado!")

# </> Abstração em Gerência de Arquivos
---
### Exemplo aplicado: Prontuários Hospitalares

### 👥 Autores
| Nome | RGM |
|---|---|
| Camilla Macedo Alves | 43713548 |
| Gabriel Oliveira de Souza | 43319327 |
| João Gabriel Rocha Cerqueira | 43912672 |
| Virgínia Mayumi Furushima de Freitas | 42114331 |

**Tema:** Abstração em Gerência de Arquivos
<br/>
**Aplicação:** Prontuários Hospitalares

**📅 Data da Apresentação:** 13/Nov/2025

**Curso:** Ciência da Computação
<br/>
**Disciplina:** Sistemas Operacionais



### Objetivo da Apresentação

Apresentar cada um dos níveis de funcionamento do sistema através das seguintes flags:

- 🔴 **Baixo Nível:** Revisar operações que ocorrem por baixo do SO.
- 🟡 **Abstração SO:** Explicar abstrações de sistemas feitos pelo Sistema Operacional.
- 🟢 **Abstração Programática:** Apresentar uma segunda camada de abstração por cima do SO através da programação.

### Tópicos Chave
0. Abstração
1. Estrutura e Diretórios
2. Permissões
3. Mecanismos de Acesso
4. Armazenamento

Caso prático: Prontuários Hospitalares

---

## 0 - Conceito de Abstração

Antes de falar sobre os meios de abstração, é importante entender o que é a abstração na computação.

### O que é?

Abstração na computação é o processo de simplificar sistemas complexos.
- Ocultar detalhes de implementação e expor apenas as funcionalidades essenciais.
- Permitir que usuários e desenvolvedores trabalhem com conceitos de alto nível, facilitando o entendimento, reutilização e manutenção do sistema.
- Baixa curva de aprendizado, sem a necessidade de conhecer todos os detalhes internos de como algo funciona para conseguir usufruir.

### Como é feito?

- 🟡 Pelo Sistema Operacional (SO):
    - Expõe chamadas simples (open, read, write) no lugar de comandos de hardware.
    - Usa drivers para traduzir pedidos genéricos para cada dispositivo físico.
    - Unifica discos e partições em um único sistema de arquivos.
    - Isola programas com processos/threads e memória virtual.
    - Aplica permissões e políticas de segurança sem que o app conheça detalhes do hardware.

- 🟢 Por Aplicativos (camada acima do SO):
    - Agrupam várias chamadas do SO em funções de alto nível (ex.: “Gerar Relatório”).
    - Usam bibliotecas/frameworks para esconder protocolos, formatos e rede.
    - Trabalham com modelos do domínio (Paciente, Consulta) em vez de pastas/arquivos brutos.
    - Impõem regras de negócio e padronizam nomes/locais de dados.
    - Oferecem UI e APIs próprias para uso simples por pessoas e outros sistemas.

Resumo em uma frase: o SO padroniza o acesso ao hardware; os apps padronizam o acesso ao problema do usuário.

---

## 1 - Estrutura e Diretórios

### 🔴 Baixo Nível

Os diretórios não ficam abaixo do sistema operacional, e sim dentro dele, na camada do sistema de arquivos, que é quem realmente organiza e controla os dados no disco.

- Os inodes são criados.

- Os blocos de dados são alocados.

- As entradas de diretório (nome → inode) são gravadas no disco.

Essa parte conversa diretamente com o hardware de armazenamento (HD, SSD, pendrive, etc).

---

### 🟡 Abstração do SO

- O sistema de arquivos é organizado em forma de árvore.

- O topo da árvore é o diretório raiz (“/”).

- Cada diretório pode conter arquivos e outros diretórios (subdiretórios).

- Os diretórios criam uma estrutura hierárquica que facilita o acesso e organização.

- Um diretório é, internamente, um arquivo especial que armazena nomes e referências para outros arquivos.

Estrutura do diretório raiz `prontuarios/` na visualização do OS:

In [ ]:
list_directory_tree("prontuarios/")

---

### 🟢 Abstração Programática

##### Representação de cores
- **🟦 Azul:** Diretório / Pasta
- **⬛ Cinza:** Arquivo

#### 1º Nível Hierárquico

Estrutura completa no diretório raiz `prontuarios/`:

In [ ]:
draw_tree_auto("prontuarios/")

A Hierarquia Nº1 é composta por duas ramificações a partir do diretório raiz `prontuarios/`, sendo elas:

- `pacientes_arquivados/`

- `pacientes_ativos/`

#### 2º Nível Hierárquico

Estrutura no diretório `pacientes_arquivados/`:

In [ ]:
draw_tree_auto("prontuarios/pacientes_arquivados/")

Estrutura no diretório `pacientes_ativos/`:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos/")

A Hierarquia Nº2 é composta por uma ramificação para cada paciente **ativo** ou **arquivado**.

O nome do diretório é composto pelo índice e nome do paciente, no seguinte formato:
``` js
"P" + index + "-" + name
```

#### 3º Nível Hierárquico

Estrutura no diretório de um paciente:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos/P3-Maria_Hipotetica/")

A Hierarquia Nº3 é composta por dois tipos de elementos: o subdiretório `profile/` contendo a imagem de identificação do paciente e subdiretórios para diferentes tipos de prontuários relacionados ao mesmo.

Os subdiretórios podem variar de acordo com o ciclo hospitalar do paciente, como:

- `consultas/`

- `exames/`

- `tratamentos/`

- `cirurgias/`

- `laboratoriais/`

- etc...

---

## 2 - Permissões

### 🔴 Baixo Nível

- No baixo nível, as permissões são apenas bits e identificadores armazenados nos metadados do arquivo (inode ou registro de diretório) pelo sistema de arquivos.

- O hardware do disco não entende permissões; ele só armazena os dados.

- O controle real de acesso só acontece quando o SO interpreta esses bits ao tentar acessar o arquivo.

---

### 🟡 Abstração do SO

As permissões de arquivos são um mecanismo do sistema operacional para controlar quem pode acessar ou modificar arquivos e diretórios. Cada arquivo tem um usuário proprietário e um grupo, e as permissões são divididas em três níveis (usuário, grupo e outros), permitindo diferentes tipos de acesso e garantindo segurança e compartilhamento adequado.

Sistemas de arquivos como o NTFS (New Technology File System) organizam e controlam o armazenamento lógico dos dados, ocultando a complexidade do hardware. No NTFS, permissões são aplicadas a todos os arquivos e pastas do volume, normalmente herdadas da pasta raiz, mas essa herança pode ser desativada. As permissões NTFS valem para acessos locais e remotos, e incluem níveis como Leitura, Execução, Gravação, Modificação, Listagem e Controle Total.

![image1](assets/permission_1.jpeg)

Além dessas, há um conjunto de permissões avançadas, que detalham ainda mais os níveis de acesso, permitindo configurações mais específicas conforme o tipo de objeto (arquivo ou pasta). Essas permissões avançadas oferecem maior granularidade e controle sobre as ações que cada usuário pode realizar.

![image2](assets/permission_2.jpeg)

Outro tipo de controle importante são as permissões de compartilhamento, aplicadas apenas a pastas compartilhadas em rede. Elas entram em vigor quando uma pasta é acessada remotamente e determinam o nível de acesso permitido sobre o conteúdo compartilhado. Embora sejam menos detalhadas do que as permissões NTFS, as permissões de compartilhamento também oferecem três níveis principais: Leitura, Modificação e Controle Total.

![image3](assets/permission_3.jpeg)

Assim, dentro da abstração de gerência de arquivos, o sistema operacional utiliza mecanismos como o NTFS e suas permissões para oferecer ao usuário uma forma segura, flexível e intuitiva de interagir com os dados, sem a necessidade de lidar diretamente com a complexidade física do armazenamento.

---

### 🟢 Abstração Programática

Organizações utilizam softwares de gerenciamento de permissões que adicionam camadas sobre o SO para facilitar controle e auditoria em larga escala:

**Exemplos de soluções populares:**

<br/>
<img src="assets/active_directory.svg" alt="Varonis" height="60"/>

- **Active Directory (Microsoft):** gerenciamento centralizado de usuários, grupos e políticas de acesso em domínios Windows.

<br/>
<img src="assets/varonis.png" alt="Varonis" height="50"/> 

- **Varonis:** plataforma para análise, auditoria e proteção de dados com foco em detecção de anomalias e conformidade.

<br/>
<img src="assets/sailpoint.png" alt="Varonis" height="50"/> 

- **SailPoint IdentityIQ:** solução enterprise para governança de identidades, auditoria e controle de acessos.

<br/>

**Pontos fortes em comparação com o SO puro:**
- **Visão centralizada:** interface única para gerenciar permissões em múltiplos servidores, aplicações e sistemas de arquivos.

- **Auditoria e compliance:** logs detalhados, relatórios de quem tem acesso a quê, e alertas de mudanças críticas.

- **Automação:** workflows de aprovação, provisionamento/revogação automática de acessos e integração com RH (onboarding/offboarding).

- **Análise de risco:** detecção de permissões excessivas, usuários com privilégios desnecessários e padrões anômalos de acesso.

- **Políticas de negócio:** aplicação de regras por departamento, cargo ou projeto, além do modelo simples rwx do SO.

---

## 3. Mecanismo de Acesso

### 🔴 Baixo Nível

No baixo nível, acessar arquivos envolve:

- Leitura e gravação de blocos diretamente no disco físico.

- Comunicação com o hardware via controladores e drivers (ex: SATA, NVMe).

- Uso de memória temporária (buffer cache) para acelerar operações.

- Sinais do disco (interrupções) para avisar o sistema quando uma operação termina.

- Métodos como o DMA (Direct Memory Access) permitem que dados sejam transferidos entre disco e RAM sem ocupar a CPU o tempo todo.

---

### 🟡 Abstração do SO

Syscalls open/read/write/close, descritores de arquivo e resolução de caminho via VFS (Virtual File System) padronizam o acesso e isolam a aplicação dos detalhes físicos.

Os mecanismos de acesso definem como os processos interagem com arquivos, abstraindo a complexidade do armazenamento físico através de interfaces padronizadas.

Nesta camada, descritores de arquivo, caminhos e chamadas de sistema (open, read, write, close) formam um CONTRATO estável que nosso trabalho reutiliza para oferecer operações de alto nível (listar, buscar, mapear) sem expor detalhes de baixo nível.

#### 🚧 Pontos fracos do acesso manual de arquivos

-  **Escalabilidade limitada:** Com o crescimento do volume de arquivos, navegar manualmente pela estrutura de diretórios torna-se cada vez mais demorado e propenso a erros

- **Busca ineficiente:** Localizar um prontuário específico requer conhecer exatamente sua localização na hierarquia de pastas, sem suporte para busca por conteúdo ou metadados

- **Inconsistência de nomenclatura:** Sem padronização automatizada, diferentes usuários podem criar arquivos com nomes inconsistentes, dificultando a organização

- **Dificuldade de integração:** Sistemas que dependem de acesso manual não se integram facilmente com outras ferramentas (relatórios, alertas, backup automatizado)

---

### 🟢 Abstração Programática

#### Abstraindo ainda mais o processo de acesso

Ao subir camadas de abstração, conseguimos mitigar os pontos fracos do gerenciamento manual de arquivos. Gerar a estrutura em um arquivo CSV adiciona uma camada estruturada de metadados que facilita consumo por outros sistemas.

Mapeamento de arquivos em formato tabular (CSV):

In [ ]:
# Listar todos os arquivos dos prontuários
save_and_display_csv("prontuarios", "mapeamento_prontuarios.csv")

Benefícios práticos do CSV gerado:


- **Visão unificada e filtrável:** status (ativo/arquivado), paciente, categoria, tipo, extensão, tamanho e caminho em um único lugar

- **Busca e filtros rápidos:** encontrar arquivos por extensão (.pdf, .jpg), por categoria (exames, consultas) ou por tamanho (arquivos grandes)

- **Integração fácil:** abre no Excel/Google Sheets, alimenta dashboards (BI) e scripts Python (Pandas)

- **Auditoria de mudanças:** comparar versões do CSV no controle de versão para ver adições/remoções e reorganizações de pastas

- **Automação e relatórios:** gerar KPIs (total por categoria, espaço ocupado por paciente, top N arquivos maiores)

- **Detecção de inconsistências:** nomes fora do padrão, arquivos em pastas erradas, extensões inesperadas

- **Portabilidade e compartilhamento:** compartilhar o mapa da estrutura sem expor o conteúdo sensível dos arquivos

- **Base para indexação:** serve como entrada para mecanismos de busca/indice sem varrer o disco toda hora

#### Mini relatório: quantidade de arquivos por categoria

Este gráfico mostra quantos arquivos existem em cada categoria (consultas, exames, etc.). Ele ilustra como o CSV permite gerar estatísticas rápidas sem navegar manualmente pelas pastas.

In [ ]:
gerar_relatorio_contagem_categoria()

---

## 4. Armazenamento

### 🔴 Baixo Nível

No baixo nível, o armazenamento de arquivos em disco é organizado através de estruturas de dados que determinam como localizar e acessar os blocos físicos onde os dados estão gravados. Uma das técnicas mais eficientes é a **alocação indexada**, que utiliza blocos de índice para mapear a localização dos dados no disco.

#### 📋 Alocação Indexada com Lista de Índices

Diferente de outras técnicas, como a alocação encadeada (lista encadeada), a **lista com índice** resolve problemas como o "acesso lento" (percorrer bloco por bloco até encontrar um determinado bloco). 

#### ⚙️ Como Funciona?

**Bloco de Índice:**
* Não armazena dados do arquivo, mas sim **metadados** (tamanho do arquivo, permissões, proprietários) e principalmente uma **lista de endereços** que apontam para os blocos de dados.

**Bloco de Dados:**
* Contém o conteúdo real do arquivo. Exemplos: imagem, prontuários de pacientes, documentos médicos.

#### 📖 Mecanismo de Leitura

Para acessar um bloco específico (exemplo: 20º bloco), o processo é dividido em **duas leituras**:

**1º Leitura:**
1. **Localizar o Índice:** O SO encontra o endereço do bloco índice no disco
2. **Carregar para RAM:** O Sistema Operacional realiza a primeira leitura para copiar o bloco de índice (do disco) para a memória RAM

**Objetivo:** O índice agora está na memória principal, pronto para consulta rápida.

**2º Leitura:**
* O SO acessa diretamente a 20ª entrada na lista do bloco de índice (que está na RAM)
* O SO extrai o endereço físico do disco (ex: bloco 415) armazenado naquela posição
* Realiza a segunda leitura para buscar o bloco de dados correspondente

**Objetivo:** O SO tem a coordenada exata de onde estão os dados no disco.

#### ✅ Vantagens:

* **Acesso Rápido (Aleatório):** Permite que o SO encontre qualquer bloco de dados do arquivo, independente do tamanho, utilizando apenas duas leituras
* **Sem Fragmentação Externa:** Diferente da alocação contígua, os blocos podem estar espalhados no disco
* **Crescimento Flexível:** É possível alocar novo bloco de dados em qualquer lugar do disco e adicionar seu endereço ao bloco de índice

#### 🚧 Desvantagens:

* **Gasto de Espaço:** Cada arquivo, mesmo os pequenos, precisa de um bloco de índice inteiro
* **Complexidade da Indireção:** Para arquivos muito grandes, o uso de indireção multinível (índices que apontam para outros índices) torna a estrutura mais complexa de ser gerenciada pelo Sistema Operacional

---

### 🟡 Abstração do SO

O Sistema Operacional abstrai a complexidade do armazenamento físico através de ferramentas visuais e APIs que escondem os detalhes de baixo nível.

#### 🗂️ Gerenciador de Arquivos

O **Gerenciador de Arquivos** é a interface gráfica do SO para manipulação de arquivos e diretórios:

<br/>
<img src="assets/arquivos.png" alt="Varonis" height="300"/> 

**Principais Funcionalidades:**
* Navegação em hierarquia de pastas
* Operações básicas: criar, copiar, mover, renomear, excluir
* Visualização de metadados (tamanho, data, permissões)
* Busca por nome, tipo ou conteúdo
* Integração com aplicativos do sistema

**Exemplos:** Windows Explorer, macOS Finder, Linux Nautilus

#### 🎯 O que o SO Abstrai:

* Localização física dos dados no disco (blocos, setores)
* Sistema de arquivos utilizado (NTFS, ext4, FAT32)
* Técnicas de alocação (contígua, encadeada, indexada)
* Operações de I/O e comunicação com hardware

**Resultado:** O usuário manipula arquivos de forma intuitiva (arrastar e soltar), sem conhecer detalhes técnicos do armazenamento físico.

---

### 🟢 Abstração Programática

Funções de gerenciamento avançado da estrutura de prontuários:

#### Exemplos de uso:

##### Criar um novo paciente (com imagem placeholder no profile/):

In [ ]:
gp.criar_paciente("Vini_Junior", categorias=['consultas', 'exames', 'cirurgias'])

Listar pacientes ativos:

In [ ]:
gp.listar_pacientes(arquivado=False)

Visualizar árvore de pacientes ativos atualizada:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos")

<br/>

##### Arquivar o paciente pelo ID (ex: arquivar P5):

In [ ]:
gp.arquivar_paciente(5)

Listar pacientes arquivados:

In [ ]:
gp.listar_pacientes(arquivado=True)

Visualizar árvore de pacientes arquivados atualizada:

In [ ]:
draw_tree_auto("prontuarios/pacientes_arquivados")

<br/>

##### Desarquivar o paciente pelo ID (ex: desarquivar P5):

In [ ]:
gp.desarquivar_paciente(5)

Listar pacientes ativos:

In [ ]:
gp.listar_pacientes(arquivado=False)

Visualizar árvore de pacientes ativos atualizada:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos")

<br/>

##### Criar um documento para o paciente (ID 4, categoria 'consultas')

In [ ]:
gp.criar_documento(
    id_paciente=4, 
    categoria='consulta', 
    nome_arquivo="consulta_2025-11-13.txt",
    conteudo="Consulta de rotina.\nPressão: 120/80\nPeso: 75kg\n")

In [ ]:
# Criar outro exame para paciente (ID 4, categoria 'exames')
gp.criar_documento(
    id_paciente=4,
    categoria='exames',
    nome_arquivo="hemograma_2025-11-13.txt",
    conteudo="Hemograma completo normal.\nLeucócitos: 7000/mm3\n")

Visualizar árvore do Paciente 4 atualizada:

In [ ]:
draw_tree_auto("prontuarios/pacientes_ativos/P4-Allexandre")

---

## 🎯 Conclusão

A abstração em gerência de arquivos é fundamental para a usabilidade dos sistemas computacionais. Através do estudo de caso com prontuários hospitalares, demonstramos como três camadas de abstração transformam operações complexas de hardware em interfaces intuitivas.

### 🔑 Vantagens da Abstração:

* **Simplicidade:** Usuários manipulam arquivos sem conhecer detalhes técnicos de hardware
* **Portabilidade:** Mesmo código funciona em diferentes sistemas de arquivos e dispositivos
* **Manutenibilidade:** Mudanças no hardware não afetam aplicações existentes
* **Produtividade:** Desenvolvedores focam na lógica de negócio, não em detalhes de I/O

### 💡 Reflexão Final

A abstração permite que profissionais de saúde foquem no atendimento, desenvolvedores criem sistemas eficientes, e o SO gerencie a complexidade do armazenamento. **A abstração não esconde apenas complexidade — ela libera potencial humano para resolver problemas mais importantes.**

---

### 🙏 Obrigado!

**Disciplina:** Sistemas Operacionais  
**Curso:** Ciência da Computação  
**Data:** 13 de Novembro de 2025

**Grupo:**
- Camilla Macedo Alves
- Gabriel Oliveira de Souza
- João Gabriel Rocha Cerqueira
- Virgínia Mayumi Furushima de Freitas